In [1]:
import pandas as pd
import random
import re
dataset=pd.read_excel('ruleBased.xlsx')
import arabicstopwords.arabicstopwords as asw
from nltk import word_tokenize
import string
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, LeaveOneOut, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
dataset

,text,label
0,لا حضرتك راجع الإعداد فى العالم حتلاقيها أكبر ...,شات
1,: كلام.المجلس.الاعلى.للخيانه والخرفانيه ينفع.ع...,شات
2,لا مش بتحبه ..ومش ممكن تخسر صديقة عمرها بنت عمها,شات
3,شكرا ويارب كل الناس تكون بخير وربنا يرفع عنا و...,شات
4,صح أخى عبدالله 👌👍,شات
...,...,...
9493,امين امين 🌹🌹🌹,شات
9494,ياااانهااااارررر ااااسود...الله يخربيت امريكا😡😡,شات
9495,تسلملي استاذ حسين🌹🌹,شات
9496,ورحمة الله. بالله عليكم اساتذتنا الكرام. استف...,اسره


In [2]:
# shuffling code

# df = pd.DataFrame(dataset)
# df = df.sample(frac=1, random_state=42)
# print(df)
# df=pd.DataFrame(df)
# df.to_excel('ruleBased.xlsx', index=False)
dataset.label=dataset.label.replace('اسره','قضيه')
dataset.label=dataset.label.replace('جنح','قضيه')
dataset.label=dataset.label.replace('مدني','قضيه')
set(dataset.label.values)

{'شات', 'قضيه'}

In [3]:
def remve_noise(cases):
    corpus=[]
    for case in cases:
        s=case.lower()
        s=re.sub("[0-9]",'',s)
        words=word_tokenize(s)
        word=[w for w in words if w not in asw.stopwords_list() if w not in string.punctuation]
        s=" ".join(word)
        corpus.append(s)
    return corpus

In [4]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(remve_noise(dataset['text']))
X = tfidf_vectorizer.transform(remve_noise(dataset['text']))
Y=dataset['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2)

In [17]:
# pd.DataFrame(X.toarray(),columns=sorted(tfidf_vectorizer.vocabulary_.keys()))

In [6]:
from sklearn import svm
from sklearn.svm import LinearSVC
sig = svm.SVC(kernel='sigmoid').fit(X_train, Y_train)

liner_SVM = LinearSVC()
liner_SVM.fit(X_train,Y_train)

LinearSVC()

In [7]:
sig_pred = sig.predict(X_test)

linersvm_pred=liner_SVM.predict(X_test)

In [8]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [9]:
print(confusion_matrix(Y_test,sig_pred))

[[1483    5]
 [  17  395]]


In [10]:
print(confusion_matrix(Y_test,linersvm_pred))

[[1484    4]
 [  21  391]]


In [11]:
print("accuracy is as sig ",accuracy_score(Y_test, sig_pred))
print("accuracy is as linear SVM ",accuracy_score(Y_test, linersvm_pred))

accuracy is as sig  0.988421052631579
accuracy is as linear SVM  0.9868421052631579


In [12]:
print(classification_report(Y_test,sig_pred))

              precision    recall  f1-score   support

         شات       0.99      1.00      0.99      1488
        قضيه       0.99      0.96      0.97       412

    accuracy                           0.99      1900
   macro avg       0.99      0.98      0.98      1900
weighted avg       0.99      0.99      0.99      1900



In [13]:
print(classification_report(Y_test, linersvm_pred))

              precision    recall  f1-score   support

         شات       0.99      1.00      0.99      1488
        قضيه       0.99      0.95      0.97       412

    accuracy                           0.99      1900
   macro avg       0.99      0.97      0.98      1900
weighted avg       0.99      0.99      0.99      1900



In [14]:
import pickle
pickle.dump(sig, open('ruleBased.pkl', 'wb'))
pickle.dump(tfidf_vectorizer, open('ruleBasedTokenizer.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
def ruleBased(text):
    text=[text]
    transform=tfidf_vectorizer.transform(remve_noise(text))
    pickled_model=pickle.load(open('ruleBased.pkl','rb'))
    return(pickled_model.predict(transform))

In [16]:
ruleBased('مال')

array(['شات'], dtype=object)